#Pecan validation analysis

Here we perform analysis on the model vs ground truth data for thermal and occupancy consumption. We contrast two models:
* ground truth thermal response: a locally-weighted regression model using separately-monitored HVAC and temperature
* the HMM thermal model

In [4]:
rm(list = ls())
options(error = recover)
library('segmented')
library('lubridate')
library('ggplot2')

In [ ]:
setwd("~/EnergyAnalytics/")
source('../../thermal_profiles/validator/metrics.r')
source('../../thermal_profiles/validator/plot_metrics.r')
source('../../utils/select_data.r')

In [2]:
DATA_PATH = '~/S3L_server/energy-data/pecan_street/usage-select/'
DUMP_PATH = '~/S3L_server/energy-data/pecan_street/models_new/'
PLOT_PATH = '~/S3L_server/plots/pecan-street-new/'
MODEL_PATH= '~/S3L_server/energy-data/pecan_street/models_new/'